<a href="https://colab.research.google.com/github/CosiMichele/uadl-cv/blob/main/workshop-content/uadl-cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer Vision: Image Analysis

In this notebook, we cover the basics of Computer Vision, first by showing a classical approach using OpenCV, then through a modern take using CNN. To help with the tutorial, we will be using the following images. 

`apples_clear.png`
<div>
<img src="https://www.kew.org/sites/default/files/styles/social/public/2022-05/apple%20cultivars.png" width="300"/>
</div>


`apples_board.png`
<div>
<img src="https://media.post.rvohealth.io/wp-content/uploads/2020/09/health-benefits-of-apples-1200x628-facebook-1200x628.jpg" width="300"/>
</div>

`apples_table.png`
<div>
<img src="https://i0.wp.com/blog.blueapron.com/wp-content/uploads/2020/09/fall_apple_varieties-1.jpg?fit=1920%2C1228&ssl=1" width="300"/>
</div>

## Table of contents

- Prerequisites
- A Classical Approach: OpenCV
- OpenCV Exercise
- A Modern Approach: CNN

## Prerequisites

Download the images you require for this exercise

In [ ]:
# Downloading images 
!wget -O apples_clear.png https://www.kew.org/sites/default/files/styles/social/public/2022-05/apple%20cultivars.png?h=1d19dfc9&itok=NT2TwltK
!wget -O apples_board.png https://media.post.rvohealth.io/wp-content/uploads/2020/09/health-benefits-of-apples-1200x628-facebook-1200x628.jpg
!wget -O apples_table.png https://i0.wp.com/blog.blueapron.com/wp-content/uploads/2020/09/fall_apple_varieties-1.jpg?fit=1920%2C1228&ssl=1

Load all the libraries necessary. This notebook is designed to run seamlessly on both Colab and your personal computer. The necessary libraries will be loaded in the following cell, accompanied by a note specifying which libraries are required for each section.

In [ ]:
# Detect if notebook is running on Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    
# Load remaining required packages
## Packages required for the classical approach
import cv2
import numpy as np

## Packages required for the modern approach
import torch
import torchvision
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image, ImageDraw
import requests
from io import BytesIO

if IN_COLAB:
    # Code for Colab environment
    from google.colab.patches import cv2_imshow
    print("All the libraries have loaded for a colab system.")
else:
    # Code for local environment
    print("All the libraries have loaded for your local system.")

## A Classical Approach: OpenCV

In this section we are going to be using OpenCV to count apples.  To achieve this, OpenCV is used in such a manner where the edges of an object are detected first and then the object is counted. This process involves the following steps:

1. Convert image to black and white. Since our goal is to count still objects in an image, the conversion to black and white helps with removing not needed features (colors).
2. Blurring the image using Gaussian Blur: this helps with the reduction of noise in the image
3. Finding the edges of the blurred objects using the Canny edge detector
4. Finding and counting the contours of the objects

In [ ]:
# import cv2                                    # These are the required libraries for this section
# import numpy as np                            # If running on your computer, all you need is cv2 and np!
# from google.colab.patches import cv2_imshow   # If you're running on colab, you will also need this

# 0. Load the image of apples
image = cv2.imread('apples_clear.png')

# Visualize image if needed 
if IN_COLAB:
    cv2_imshow(image)
else:
    cv2.imshow(image)

In [ ]:
# 1. Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Visualize image if needed 
if IN_COLAB:
    cv2_imshow(gray)
else:
    cv2.imshow(gray)

In [ ]:
# 2. Apply Gaussian blur to reduce noise
ksizeX = 7
ksizeY = 7
blurred = cv2.GaussianBlur(gray, (ksizeX, ksizeY), 0)

# 3. Perform edge detection
edges = cv2.Canny(blurred, 40, 100)

# Visualize image if needed 
if IN_COLAB:
    cv2_imshow(edges)
else:
    cv2.imshow(edges)

In [ ]:
# 4. Find contours in the edge-detected image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Initialize a counter for the number of apples
apple_count = 0

# Iterate through the contours and filter out small ones (potential noise)
valid_contours = []
for contour in contours:
    if cv2.contourArea(contour) > 50:
        valid_contours.append(contour)
        apple_count += 1
valid_contours = np.asarray(valid_contours, dtype=object)

# Print the number of apples found
print(f"Number of apples: {apple_count}")

# Visualize the countours
show_contours = np.stack((edges, edges, edges), axis=-1)

for contour in valid_contours:
  for [point] in contour:
    show_contours[point[1], point[0], :] = [0, 0, 255]

# Visualize image if needed 
if IN_COLAB:
    cv2_imshow(show_contours)
else:
    cv2.imshow(show_contours)


## OpenCV Exercise

Try it out! See if you can count apples using the other images (`apples_board.png` or `apples_table.png`, the latter is **strongly** recommended)!

Following are the cells provided for the exercise. 

In [ ]:
# 0. Load the image of apples
image = cv2.imread('<image_you_want_to_load.png>')

# Visualize image if needed 
if IN_COLAB:
    cv2_imshow(image)
else:
    cv2.imshow(image)

In [ ]:
# 1. Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Visualize image if needed 
if IN_COLAB:
    cv2_imshow(gray)
else:
    cv2.imshow(gray)

In [ ]:
# 2. Apply Gaussian blur to reduce noise
ksizeX = 7                                              # These two values are the kerlel size: these can be negative, but these have to be an odd number 
ksizeY = 7                                              # The larger the numbers, the higher the smoothing effect is applied
blurred = cv2.GaussianBlur(gray, (ksizeX, ksizeY), 0)   # 0 is the value of sigmaX: the standard deviation of the Gaussian kernel in the X direction. If it is set to 0, OpenCV calculates it based on the kernel size. 

# Visualize image if needed 
if IN_COLAB:
    cv2_imshow(blurred)
else:
    cv2.imshow(blurred)

In [ ]:
# 3. Perform edge detection
edges = cv2.Canny(blurred, 40, 100) # You may find it useful to change these values
                                    # The second value is the low threshold, the third is the high threshold
                                    # the low or high values are tied to the intensity of the edges
# Visualize image if needed 
if IN_COLAB:
    cv2_imshow(edges)
else:
    cv2.imshow(edges)

In [ ]:
# 4. Find contours in the edge-detected image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Initialize a counter for the number of apples
apple_count = 0

# Iterate through the contours and filter out small ones (potential noise)
valid_contours = []
for contour in contours:
    if cv2.contourArea(contour) > 50:                       # You can play around with this value to establish what a contour is                                  
        valid_contours.append(contour)
        apple_count += 1
valid_contours = np.asarray(valid_contours, dtype=object)

# Print the number of apples found
print(f"Number of apples: {apple_count}")

# Visualize the countours
show_contours = np.stack((edges, edges, edges), axis=-1)

for contour in valid_contours:
  for [point] in contour:
    show_contours[point[1], point[0], :] = [0, 0, 255]

# Visualize image if needed 
if IN_COLAB:
    cv2_imshow(show_contours)
else:
    cv2.imshow(show_contours)

# A Modern Approach: CNN

As you may have figured out by this point, using OpenCV to count objects isn't simple and not always reliable. Modern methods like CNN can help speed up the application process, however these do require training. 

In this next section we are going to be using pre-trained weights, meaning that we do not need to train a model.

The steps that are taken here are:

1. Loading the model and setup preprocessing of images
2. Write a function to perform detection on an image using torch and the pre-trained model
3. Write a function that creates boxes and labels
4. Wrote a function that shows the results

In [ ]:
# import torch                                                          # These are the libraries required for this section
# import torchvision
# from torchvision import transforms
# from torchvision.models.detection import fasterrcnn_resnet50_fpn
# from PIL import Image, ImageDraw
# import requests
# from io import BytesIO

# Load the pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Modify the transformation for input images
transform = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
# Function to perform object detection on an image
def detect_objects(model, image, threshold=0.5):
    # Transform the input image
    input_image = transform(image).unsqueeze(0)

    # Perform inference
    with torch.no_grad():
        prediction = model(input_image)

    # Filter out predictions below the threshold
    boxes = prediction[0]['boxes'][prediction[0]['scores'] > threshold]
    scores = prediction[0]['scores'][prediction[0]['scores'] > threshold]
    labels = prediction[0]['labels'][prediction[0]['scores'] > threshold]

    return boxes, scores, labels

In [ ]:
# Function to draw bounding boxes and labels on the image
def draw_boxes(image, boxes, scores, labels):
    draw = ImageDraw.Draw(image)
    for box, score, label in zip(boxes, scores, labels):
        draw.rectangle([box[0], box[1], box[2], box[3]], outline="red", width=3)
        draw.text((box[0], box[1]), f"Label: {label}, Score: {score:.2f}", fill="red")
    return image

In [ ]:
# Function to visualize the detection results
def visualize_results(image, boxes, scores, labels):
    image_with_boxes = draw_boxes(image, boxes, scores, labels)
    display(image_with_boxes)

In [ ]:
# Example image path
image_path = "apples_table.png"

# Load image and perform object detection
image = Image.open(image_path)

boxes, scores, labels = detect_objects(model, image, threshold=0.5)

# Visualize the results
visualize_results(image, boxes, scores, labels)